In [195]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta as td
import calendar
from dateutil.relativedelta import relativedelta

In [196]:
df = pd.read_csv('template.csv')
df[['Invoice Date','Paid Date']] = df[['Invoice Date','Paid Date']].apply(pd.to_datetime, format='%m/%d/%Y')

In [197]:
# Assume due date for all is Invoice Date + 30 days
df['Due Date'] = df['Invoice Date'] + td(days=30)

In [198]:
def ARaging(ARlog, calcDate):
    ################################################
    '''Function takes the ARlog of a customer and returns AR aging report as it would have looked on calcDate'''
    
    ARaged = ARlog.loc[(ARlog['Invoice Date']<calcDate) & (ARlog['Paid Date']>calcDate)].copy() # all invoices issued before date "calcDate" and not paid until date "calcDate". 
    ARaged.loc[:,'Days Remaining']=(ARaged['Due Date'] - calcDate).dt.days

    # create a list of our conditions
    conditions = [
        (ARaged.loc[:,'Days Remaining'] <= 0) & (ARaged.loc[:,'Days Remaining'] >= -30),
        (ARaged.loc[:,'Days Remaining'] < -30) & (ARaged.loc[:,'Days Remaining'] >= -60),
        (ARaged.loc[:,'Days Remaining'] < -60) & (ARaged.loc[:,'Days Remaining'] >= -90),
        (ARaged.loc[:,'Days Remaining'] < -90) & (ARaged.loc[:,'Days Remaining'] >= -120),
        (ARaged.loc[:,'Days Remaining'] < -120), # if payment is overdue 120 days then it is put under Bad Debt
        (ARaged.loc[:,'Days Remaining'] >= 0), # represents current invoices. Invoices that are not due yet
        ]

    # list of the values we want to assign for each condition
    values = ['1','2','3','4','5','0']

    # create a new column and assign values to it using the lists "conditions" and "values" as arguments
    ARaged.loc[:,'status'] = np.select(conditions, values)
    ARaged = ARaged.groupby(by=['status']).sum().reset_index()
    del(ARaged['Days Remaining'])

    missing = list(set(['0','1','2','3','4','5']) - set(ARaged.status.drop_duplicates()))
    missing = [int(ele) for ele in missing]; missing.sort()
    missing = [list(ele) for ele in zip(missing,[0]*len(missing))] 
    missing = pd.DataFrame(missing,columns=['status','Amount'])

    ARaged = ARaged.append(missing)
    ARaged.loc[:,'status'] = ARaged['status'].astype(int) 
    ARaged = ARaged.sort_values(by=['status']).reset_index(); del(ARaged['index']); del(ARaged['status'])
    ARaged = ARaged.T
    ARaged['calcDate'] = calcDate
    return(ARaged)

In [199]:
def reporting(df_uploaded):
    GrandARaging = pd.DataFrame([])
    for cust in df_uploaded['Customer'].drop_duplicates():
        df_uploaded_cust = df_uploaded[df_uploaded['Customer']==cust].copy()

        i = min(df_uploaded_cust['Invoice Date'])
        i = i + relativedelta(day=31)

        #counter = 0
        while i < max(df_uploaded_cust['Invoice Date']):
            #counter = counter  + 1 
            #print(i)
            i = i + relativedelta(months=+1)
            i = i + relativedelta(day=31)
            temp1 = ARaging(df_uploaded_cust,i); temp1['customer'] = cust
            
            ### Calculating Payments Recieved in the month at the end of which AR aging calculations are made.### 
            j = i - relativedelta(months=+1)
            j = j - relativedelta(day=31)
            temp2 = df_uploaded_cust.loc[(df_uploaded_cust['Paid Date']<i) & (df_uploaded_cust['Paid Date']>j) ].copy() ## dataframe of all payments made in month of calcDate
            temp1['ActualCashFlowIn'] = sum(temp2['Amount'])
            #####################################################################################################

            GrandARaging = GrandARaging.append(temp1)
            #GrandARaging = GrandARaging.reset_index()
            del(temp1); del(temp2)

    GrandARaging = GrandARaging.reset_index()
    del(GrandARaging['index'])
    return(GrandARaging)

In [200]:
def calcProbs(ARagingReport):
    s1ToP = []
    s2ToP = []
    s3ToP = []
    s4ToP = []
    
    s1ToP = (ARagingReport[1] - ARagingReport[2].shift(-1))/ARagingReport[1]
    s2ToP = (ARagingReport[2] - ARagingReport[3].shift(-1))/ARagingReport[2]
    s3ToP = (ARagingReport[3] - ARagingReport[4].shift(-1))/ARagingReport[3]
    s4ToP = (ARagingReport[4] - ARagingReport[5].shift(-1))/ARagingReport[4]
    
    return(s1ToP,s2ToP,s3ToP,s4ToP)

In [215]:
def forecast(df_uploaded):
    GrandARaging = reporting(df)
    aggGrandARaging = GrandARaging.groupby('calcDate').sum()

    # s1ToP, s2ToP,..,s4ToP are equaivalent to T1, T2,...,T4 in Corcoran
    s1ToP, s2ToP, s3ToP, s4ToP = calcProbs(aggGrandARaging)
    aggGrandARaging['s1ToP'] = s1ToP.shift(1)
    aggGrandARaging['s2ToP'] = s2ToP.shift(1)
    aggGrandARaging['s3ToP'] = s3ToP.shift(1)
    aggGrandARaging['s4ToP'] = s4ToP.shift(1)
    aggGrandARaging = aggGrandARaging.reset_index()
    
    # We assume stage 5 is bad payments. I.e., once account payable goes to stage 5, no payment is coming back 
    # Here, we are calculating exponential forecasts of transition probabilities
    aggGrandARaging['expProb_s1ToP'] = aggGrandARaging['s1ToP'].ewm(alpha=0.1, adjust=False).mean()
    aggGrandARaging['expProb_s2ToP'] = aggGrandARaging['s2ToP'].ewm(alpha=0.1, adjust=False).mean()
    aggGrandARaging['expProb_s3ToP'] = aggGrandARaging['s3ToP'].ewm(alpha=0.1, adjust=False).mean()
    aggGrandARaging['expProb_s4ToP'] = aggGrandARaging['s4ToP'].ewm(alpha=0.1, adjust=False).mean()
    aggGrandARaging['expProb_s5ToP'] = 0*len(aggGrandARaging)
    
    # Here we are using exponential smoothing to forcast the amount of payments that would go from stage i to P in time period t
    aggGrandARaging['fore1'] = aggGrandARaging[1]*aggGrandARaging['expProb_s1ToP']
    aggGrandARaging['fore2'] = aggGrandARaging[1]*aggGrandARaging['expProb_s2ToP']
    aggGrandARaging['fore3'] = aggGrandARaging[1]*aggGrandARaging['expProb_s3ToP']
    aggGrandARaging['fore4'] = aggGrandARaging[1]*aggGrandARaging['expProb_s4ToP']
    aggGrandARaging['fore5'] = aggGrandARaging[1]*aggGrandARaging['expProb_s5ToP']
    aggGrandARaging['stochasticForecast'] = aggGrandARaging['fore1']+aggGrandARaging['fore2']+aggGrandARaging['fore3']+aggGrandARaging['fore4']+aggGrandARaging['fore5']
    return(aggGrandARaging)

In [222]:
result_corcoran = forecast(df)
result_corcoran

,calcDate,0,1,2,3,4,5,ActualCashFlowIn,s1ToP,s2ToP,...,expProb_s2ToP,expProb_s3ToP,expProb_s4ToP,expProb_s5ToP,fore1,fore2,fore3,fore4,fore5,stochasticForecast
0,2012-05-31,48734.505450,14398.244071,0.000000,0.000000,0.000000,0.000000,8119.235497,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN
1,2012-06-30,48647.894155,35897.821262,0.000000,0.000000,0.000000,0.000000,31718.359856,1.000000,NaN,...,NaN,NaN,NaN,0,35897.821262,NaN,NaN,NaN,0.0,NaN
2,2012-07-31,46884.216051,38525.108031,24275.721762,0.000000,0.000000,0.000000,21744.885623,0.323755,NaN,...,NaN,NaN,NaN,0,35919.866904,NaN,NaN,NaN,0.0,NaN
3,2012-08-31,21981.361891,39245.183225,4530.196169,209.775900,1961.300914,0.000000,71524.523152,0.882409,0.991359,...,0.991359,NaN,NaN,0,36395.153923,38906.050419,NaN,NaN,0.0,NaN
4,2012-09-30,33337.057906,19759.986685,11485.311280,4530.196169,209.775900,1961.300914,30554.259089,0.707345,0.000000,...,0.892223,0.00000,0.00,0,17890.206989,17630.309698,0.000000,0.000000,0.0,35520.516688
5,2012-10-31,47328.683768,28769.255073,3827.304338,0.000000,2629.307837,0.000000,34305.417645,0.806310,1.000000,...,0.903000,0.04196,0.10,0,25761.974627,25978.651046,1207.169386,2876.925507,0.0,55824.720567
6,2012-11-30,29442.366928,24361.199033,590.894493,0.000000,0.000000,2629.307837,40037.976277,0.979461,1.000000,...,0.912700,0.04196,0.09,0,22019.311785,22234.476810,1022.205601,2192.507913,0.0,47468.502110
7,2012-12-31,30241.931951,25245.303398,4533.877868,0.000000,0.000000,0.000000,31078.101847,0.813889,1.000000,...,0.921430,0.04196,0.09,0,22591.271434,23261.789659,1059.302972,2272.077306,0.0,49184.441371
8,2013-01-31,34068.892569,23195.902993,3821.649072,0.000000,0.000000,0.000000,33003.561153,0.848619,1.000000,...,0.929287,0.04196,0.09,0,20650.040886,21555.659166,973.309316,2087.631269,0.0,45266.640638
9,2013-02-28,29433.523693,15943.348869,3873.051311,0.000000,0.000000,0.000000,39240.236821,0.833029,1.000000,...,0.936359,0.04196,0.09,0,14102.267200,14928.692031,668.989260,1434.901398,0.0,31134.849889


In [223]:
def shapeOfYou(df_on_calcDate):
    PLT = df_on_calcDate['Paid Date'] - df_on_calcDate['Invoice Date']
    xbar = PLT.mean().days
    gamma = PLT.min().days
    shape = np.round(2*(xbar-gamma)/(np.pi)**0.5,4)
    return(shape,gamma)

In [224]:
## Calculate payment probability
def calcPayProb(df_monthly):
    probPaid = []
    for j in range(0,len(df_monthly)):
        if df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item()>= gamma:
            pp = np.round(1 - np.exp(-(2*(df_monthly['Invoice Sent days ago'][j:j+1].dt.days.item()-gamma)*df_monthly['delta t'][j:j+1].item()+df_monthly['delta t'][j:j+1].item()**2)/shape**2),4)
        elif (df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item()< gamma) and (df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item() + df_monthly[j:j+1]['delta t'].item()>= gamma):
            pp = np.round(1 - np.exp(-((df_monthly['Invoice Sent days ago'][j:j+1].dt.days.item() + df_monthly['delta t'][j:j+1].item()-gamma)**2)/shape**2),4)
        elif df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item() + df_monthly[j:j+1]['delta t'].item()<= gamma:
            pp = 0 
        probPaid.append(pp)
    return(probPaid)

def bayForecast(df_uploaded):
    custList = []
    calcDateList = []
    forecastList = []
    bayesianForecast = pd.DataFrame([])
    for cust in df_uploaded['Customer'].drop_duplicates():
        df_uploaded_cust = df_uploaded[df_uploaded['Customer']==cust].copy()
        
        i = min(df_uploaded_cust['Invoice Date'])
        i = i + relativedelta(day=31)
        
        #counter = 0
        while i < max(df_uploaded_cust['Invoice Date']):
            #counter = counter  + 1 
            #print(i)
            i = i + relativedelta(months=+1)
            i = i + relativedelta(day=31)
            temp = df_uploaded_cust.loc[(df_uploaded_cust['Invoice Date']<i)].copy()
            shape,gamma = shapeOfYou(temp)

            temp1 = temp[temp['Paid Date']>i].copy(); temp1.reset_index(drop=True) #Keep only the invoices not cleared till calcDate, i. Forecast will be made only for these invoices
            temp1['Invoice Sent days ago'] = i-temp1['Invoice Date']
            temp1['delta t'] = [30]*len(temp1)

            custList.append(cust)
            calcDateList.append(i)
            forecastList.append(sum(temp1['Amount']*calcPayProb(temp1))) #Dollars coming in
    
    bayesianForecast['Customer'] = custList
    bayesianForecast['calcDate'] = calcDateList
    bayesianForecast['bayForecast'] = forecastList
    bayesianForecast = bayesianForecast.groupby('calcDate').sum().reset_index()
    return(bayesianForecast)

In [225]:
result_pate = bayForecast(df)

In [231]:
result = result_corcoran.merge(result_pate,on='calcDate')
result = result[['calcDate','ActualCashFlowIn','stochasticForecast','bayForecast']]

beta = 0.1
result['Forecast'] = beta*result['stochasticForecast'] + (1-beta)*result['bayForecast']
result

,calcDate,ActualCashFlowIn,stochasticForecast,bayForecast,Forecast
0,2012-05-31,8119.235497,NaN,27950.227365,NaN
1,2012-06-30,31718.359856,NaN,46169.460759,NaN
2,2012-07-31,21744.885623,NaN,71474.845077,NaN
3,2012-08-31,71524.523152,NaN,45382.183921,NaN
4,2012-09-30,30554.259089,35520.516688,44829.711487,43898.792007
5,2012-10-31,34305.417645,55824.720567,45687.590633,46701.303626
6,2012-11-30,40037.976277,47468.502110,31566.707383,33156.886855
7,2012-12-31,31078.101847,49184.441371,37200.464282,38398.861991
8,2013-01-31,33003.561153,45266.640638,35576.905501,36545.879015
9,2013-02-28,39240.236821,31134.849889,28437.474927,28707.212424


In [166]:
df_uploaded_cust = df[df['Customer']=='customer3'].copy()
#print('%%%%%%%%%%%%%%%')
#print(cust)
i = min(df_uploaded_cust['Invoice Date'])
i = i + relativedelta(day=31)

while i < max(df_uploaded_cust['Invoice Date']):
            #counter = counter  + 1 
#print(i)
    i = i + relativedelta(months=+1)
    i = i + relativedelta(day=31)

In [167]:
temp = df_uploaded_cust.loc[(df_uploaded_cust['Invoice Date']<i)].copy()
shape,gamma = shapeOfYou(temp)
gamma

28

In [168]:
temp1['Invoice Sent days ago'][j:j+1]#+temp1['delta t'][j:j+1]-gamma)**2)/gamma**2

Series([], Name: Invoice Sent days ago, dtype: timedelta64[ns])

In [178]:
temp = df_uploaded_cust.loc[(df_uploaded_cust['Invoice Date']<i)].copy()
shape,gamma = shapeOfYou(temp)

temp1 = temp[temp['Paid Date']>i].copy(); temp1.reset_index(drop=True) #Keep only the invoices not cleared till calcDate, i. Forecast will be made only for these invoices
temp1['Invoice Sent days ago'] = i-temp1['Invoice Date']
#temp1['delta t'] = temp1['Due Date'] - i
temp1['delta t'] = [30]*len(temp1)

## Calculate payment probability
def calcPayProb(df_monthly):
    probPaid = []
    for j in range(0,len(df_monthly)):
        if df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item()>= gamma:
            pp = np.round(1 - np.exp(-(2*(df_monthly['Invoice Sent days ago'][j:j+1].dt.days.item()-gamma)*df_monthly['delta t'][j:j+1].item()+df_monthly['delta t'][j:j+1].item()**2)/shape**2),4)
        elif (df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item()< gamma) and (df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item() + df_monthly[j:j+1]['delta t'].item()>= gamma):
            pp = np.round(1 - np.exp(-((df_monthly['Invoice Sent days ago'][j:j+1].dt.days.item() + df_monthly['delta t'][j:j+1].item()-gamma)**2)/shape**2),4)
        elif df_monthly[j:j+1]['Invoice Sent days ago'].dt.days.item() + df_monthly[j:j+1]['delta t'].item()<= gamma:
            pp = 0 
        probPaid.append(pp)
    return(probPaid)

#Dollars coming in
sum(temp1['Amount']*calcPayProb(temp1))

10522.050370216526

In [176]:
temp1

,Customer,Invoice Date,Paid Date,Amount,Due Date,Invoice Sent days ago,delta t
831,customer3,2013-01-30,2013-04-01,215.536913,2013-03-01,60 days,30
832,customer3,2013-01-30,2013-04-01,519.764934,2013-03-01,60 days,30
834,customer3,2013-01-31,2013-04-01,430.394122,2013-03-02,59 days,30
845,customer3,2013-02-06,2013-04-01,917.790448,2013-03-08,53 days,30
851,customer3,2013-02-11,2013-04-05,703.056615,2013-03-13,48 days,30
852,customer3,2013-02-11,2013-06-14,731.361124,2013-03-13,48 days,30
853,customer3,2013-02-11,2013-06-14,210.509873,2013-03-13,48 days,30
854,customer3,2013-02-11,2013-06-14,306.075880,2013-03-13,48 days,30
855,customer3,2013-02-11,2013-06-14,313.800301,2013-03-13,48 days,30
856,customer3,2013-02-13,2013-04-05,229.801661,2013-03-15,46 days,30


In [171]:
temp1['test'] = temp1['Invoice Sent days ago'] + temp1['delta t']
temp1

,Customer,Invoice Date,Paid Date,Amount,Due Date,Invoice Sent days ago,delta t,test
831,customer3,2013-01-30,2013-04-01,215.536913,2013-03-01,60 days,-30 days,30 days
832,customer3,2013-01-30,2013-04-01,519.764934,2013-03-01,60 days,-30 days,30 days
834,customer3,2013-01-31,2013-04-01,430.394122,2013-03-02,59 days,-29 days,30 days
845,customer3,2013-02-06,2013-04-01,917.790448,2013-03-08,53 days,-23 days,30 days
851,customer3,2013-02-11,2013-04-05,703.056615,2013-03-13,48 days,-18 days,30 days
852,customer3,2013-02-11,2013-06-14,731.361124,2013-03-13,48 days,-18 days,30 days
853,customer3,2013-02-11,2013-06-14,210.509873,2013-03-13,48 days,-18 days,30 days
854,customer3,2013-02-11,2013-06-14,306.075880,2013-03-13,48 days,-18 days,30 days
855,customer3,2013-02-11,2013-06-14,313.800301,2013-03-13,48 days,-18 days,30 days
856,customer3,2013-02-13,2013-04-05,229.801661,2013-03-15,46 days,-16 days,30 days


In [123]:
temp1[j:j+1]['Invoice Sent days ago'].dt.days.item()< gamma

True

In [114]:
(temp1[j:j+1]['Invoice Sent days ago'].dt.days + temp1[j:j+1]['delta t'].dt.days)>= gamma

0    True
dtype: bool

In [121]:
if (temp1[j:j+1]['Invoice Sent days ago'].dt.days< gamma):
    print('cool')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [64]:
temp1

,Customer,Invoice Date,Paid Date,Amount,Due Date,Invoice Sent days ago,delta t
0,customer1,2012-04-16,2012-05-24,314.564020,2012-05-16,14 days,16 days
1,customer1,2012-04-17,2012-05-24,457.168136,2012-05-17,13 days,17 days
2,customer1,2012-04-17,2012-05-24,886.257560,2012-05-17,13 days,17 days
3,customer1,2012-04-18,2012-05-24,420.003653,2012-05-18,12 days,18 days
4,customer1,2012-04-18,2012-05-24,130.894654,2012-05-18,12 days,18 days
5,customer1,2012-04-19,2012-05-24,752.478774,2012-05-19,11 days,19 days
6,customer1,2012-04-19,2012-05-28,564.857112,2012-05-19,11 days,19 days
7,customer1,2012-04-20,2012-05-24,615.104889,2012-05-20,10 days,20 days
8,customer1,2012-04-20,2012-05-24,766.019031,2012-05-20,10 days,20 days
9,customer1,2012-04-23,2012-05-24,554.303957,2012-05-23,7 days,23 days
